In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 %cd /content/drive/MyDrive/colab/newdata

/content/drive/MyDrive/colab/newdata


In [3]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('Thunderbird.log_templates.csv')
df.head()

,EventId,EventTemplate,Occurrences
0,4af88f8f,NET: Registered protocol family <*>,21791
1,385a86d9,divert: not allocating divert_blk for non-ethe...,6238
2,8938d159,Server listening on <*> port <*>.,4196
3,f0db1fd2,MOSAL(<*>): mnt_projects<*> MOSAL_virt_to_phys...,1633457
4,743ed62f,crond <*> succeeded,7249


In [14]:
df = df.values
spirit = {}
EventID = df[:,0]
for i in range(len(df)):
  tmp = str(EventID[i])
  spirit[tmp] = i


In [16]:
template = df[:,1]

In [17]:
log = pd.read_csv('Thunderbird.log_structured.csv')
log.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,LineId,Label,Timestamp,Date,User,Month,Day,Time,Location,Component,PID,Content,EventId,EventTemplate,ParameterList
0,1,-,1147122219,2006.05.08,cn505,May,8,14:03:39,cn505/cn505,kernel,NaN,NET: Registered protocol family 10,4af88f8f,NET: Registered protocol family <*>,[u'10']
1,2,-,1147122219,2006.05.08,cn505,May,8,14:03:39,cn505/cn505,kernel,NaN,divert: not allocating divert_blk for non-ethe...,385a86d9,divert: not allocating divert_blk for non-ethe...,[u'sit0']
2,3,-,1147122219,2006.05.08,cn505,May,8,14:03:39,cn505/cn505,sshd,2194.0,Server listening on :: port 22.,8938d159,Server listening on <*> port <*>.,"[u'::', u'22']"
3,4,-,1147122219,2006.05.08,cn510,May,8,14:03:39,cn510/cn510,kernel,NaN,MOSAL(1): mnt_projects/sysapps/src/ib/topspin/...,f0db1fd2,MOSAL(<*>): mnt_projects<*> MOSAL_virt_to_phys...,"[u'1', u'/sysapps/src/ib/topspin/topspin-src-3..."
4,5,-,1147122219,2006.05.08,cn512,May,8,14:03:39,cn512/cn512,kernel,NaN,MOSAL(1): mnt_projects/sysapps/src/ib/topspin/...,f0db1fd2,MOSAL(<*>): mnt_projects<*> MOSAL_virt_to_phys...,"[u'1', u'/sysapps/src/ib/topspin/topspin-src-3..."


In [18]:
log = log.values
label = log[:,1]
event = log[:,12]

In [21]:
true_label = []
true_event = []
cnt = 0
for i in range(len(label)):
  if label[i][0] != '-':
    true_label.append(1)
  else:
    true_label.append(0)
  if event[i] in spirit:
    true_event.append(spirit[event[i]])
  ''' 
    else:
    print(i)
    cnt += 1
    template = np.append(template,log[i,11])
    EventID = np.append(EventID,event[i])
    spirit[event[i]] = len(spirit)
    true_event.append(spirit[event[i]])'''

In [22]:
###in ra template
#minh co cot template, eventID

print(len(template))
ID = []
for i in range(0, len(template)):
  ID.append(i)
  

4992


In [23]:
spirit_template = pd.DataFrame(columns=['ID','template'])
spirit_template['template'] = template
spirit_template['ID'] = ID

spirit_template.to_csv('Thunderbird_template.csv', index=False, header=True)

In [24]:
sequence_length = 50
sequence = []
final_label = []
tmp = ''
flag = False
for i in range(len(true_label)):
  tmp += str(int(true_event[i]))+' '
  if true_label[i] == 1:
    flag = True
  if (i+1)%sequence_length == 0:
    sequence.append(tmp)
    if (flag == True):
      final_label.append(1)
    else:
      final_label.append(0)
    tmp = ''
    flag = False

In [25]:
spirit = pd.DataFrame(columns=['sequence','label'])
spirit['sequence'] = sequence
spirit['label'] = final_label
spirit.to_csv('Thunderbird_fulllog.csv', index=False, header=True)

In [26]:
from collections import Counter
print(Counter(final_label))
print(Counter(label))

Counter({0: 198125, 1: 1058})
Counter({'-': 9954226, 'PBS_CON': 2542, 'VAPI': 1458, 'MPT': 760, 'EXT_FS': 125, 'CPU': 43, 'ECC': 5, 'PBS_BFD': 1})


normal: 4 235 109
abnormal: 764 891

##label sequence
normal: 64 611
abnormal: 35 389 

In [27]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [28]:
file = tf.keras.utils
raw_df = pd.read_csv('Thunderbird_fulllog.csv')
raw_df.head()

,sequence,label
0,0 1 2 3 3 3 3 3 3 3 3 3 3 3 3 4 5 6 7 8 9 3 3 ...,0
1,3 11 10 10 12 13 0 1 14 15 2 16 10 17 17 17 3 ...,0
2,17 17 17 17 17 17 17 17 17 17 17 17 23 24 24 2...,0
3,14 15 2 16 10 17 17 17 17 17 17 17 17 17 17 17...,0
4,28 29 30 31 31 31 31 31 31 31 32 32 32 32 32 3...,0


In [29]:
raw_df[['sequence', 'label']].describe()

,label
count,199183.000000
mean,0.005312
std,0.072688
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [30]:
neg, pos = np.bincount(raw_df['label'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 199183
    Positive: 1058 (0.53% of total)



In [32]:
cleaned_df = raw_df.copy()
train_df, test_df = train_test_split(cleaned_df, test_size=0.8)
pd.DataFrame(columns=['sequence','label'])


FileNotFoundError: ignored

In [ ]:
train_df.to_csv('../test/datasets/Thunderbird/train.csv', index = False, header = False, mode = 'a')
test_df.to_csv('../test/datasets/Thunderbird/test.csv', index = False, header = False, mode = 'a')

In [ ]:
print(Counter(train_df))
print(Counter(test_df))